# Part 1

--- Day 16: Ticket Translation ---

As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like `class: 1-3 or 5-7` means that one of the fields in every ticket is named `class` and can be any value in the ranges `1-3` or `5-7` (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

```
.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
```

Here, ? represents text in a language you don't understand. This ticket might be represented as `101,102,103,104,301,302,303,401,402,403`; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:

```
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
```

It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: `4 + 55 + 12 = 71`.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

In [1]:
(import [dataclasses [dataclass]])
(import [pathlib [Path]])

In [2]:
(setv INPUT_FILE (/ (.cwd Path) "inputs" "day16" "part1.txt"))

In [3]:
(defn input-lines []
  (with [fp (.open INPUT_FILE)]
    (yield-from (gfor line fp (.strip line)))))

In [4]:
(with-decorator dataclass
  (defclass Range []
    (^int low)
    (^int high)
    (defn within [self value]
      (and (<= self.low value) (<= value self.high)))))

In [5]:
(defn read-rule [line]
  "departure location: 29-917 or 943-952"
  (setv [label ranges] (line.split ": "))
  (setv ranges
    (tuple
      (gfor
        range_
        (ranges.split " or ")
        (Range #* (gfor value (range_.split "-") (int value))))))
  (, label ranges))

In [6]:
(read-rule "departure location: 29-917 or 943-952")

('departure location', (Range(low=29, high=917), Range(low=943, high=952)))

In [7]:
(defn read-rules [lines]
  (dict (gfor line lines (read-rule line))))

In [8]:
(defn read-ticket [line]
  "191,89,73,139,71,103,109,53,97,179,59,67,79,101,113,157,61,107,181,137"
  (tuple (gfor value (line.split ",") (int value))))

In [9]:
(read-ticket "191,89,73,139,71,103,109,53,97,179,59,67,79,101,113,157,61,107,181,137")

(191, 89, 73, 139, 71, 103, 109, 53, 97, 179, 59, 67, 79, 101, 113, 157, 61, 107, 181, 137)

In [10]:
(defn read-my-ticket [lines]
  (read-ticket (nth lines 1)))

In [11]:
(defn load-inputs [lines]
  (,
    (read-rules (take-while identity lines))
    (read-my-ticket (take-while identity lines))
    (tuple (gfor line (islice lines 2 None) (read-ticket line)))))

In [12]:
(defn num-invalid-all [num ranges]
  (all (gfor range_ ranges (not (range_.within num)))))

In [13]:
(defn invalid-values [nums ranges]
  (for [num nums] (when (num-invalid-all num ranges) (yield num))))

In [14]:
(setv TEST-RANGES [(Range 1 3) (Range 5 7) (Range 6 11) (Range 33 44) (Range 13 40) (Range 45 50)])

In [15]:
(list (invalid-values [7 3 47] TEST-RANGES))

[]

In [16]:
(list (invalid-values [38 6 12] TEST-RANGES))

[12]

In [17]:
(setv [RULES MY-TICKET TICKETS] (load-inputs (input-lines)))

In [18]:
(sum (invalid-values (flatten TICKETS) (flatten (RULES.values))))

23925

# Part 2

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:

```
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
```

Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [19]:
(import [collections [defaultdict]])

In [20]:
(defn ticket-valid [nums ranges]
  (all (gfor num nums (any (gfor range_ ranges (range_.within num))))))

In [21]:
(ticket-valid [7 3 47] TEST-RANGES)

True

In [22]:
(ticket-valid [38 6 12] TEST-RANGES)

False

In [23]:
(defn valid-tickets [tickets ranges]
  (gfor ticket tickets :if (ticket-valid ticket ranges) ticket))

In [24]:
(len (list (valid-tickets TICKETS (flatten (RULES.values)))))

190

In [25]:
(defn all-within [nums ranges]
  (all (gfor num nums (any (gfor range_ ranges (range_.within num))))))

In [26]:
(all-within [1 2 3 7 8 9] [(Range 1 3) (Range 7 9)])

True

In [27]:
(all-within [1 2 3 4 7 8 9] [(Range 1 3) (Range 7 9)])

False

In [38]:
(defn narrow-fields [possible-labels]
  (while (not (all (gfor labels (possible-labels.values) (= (len labels) 1))))
    (setv single-possible
      (sfor labels (possible-labels.values) :if (= (len labels) 1) (get labels 0)))
    (for [[index labels] 
          (gfor [index labels] (possible-labels.items) :if (> (len labels) 1) [index labels])]
      (assoc possible-labels index (tuple (remove (fn [value] (in value single-possible)) labels)))))
  (dfor [index labels] (possible-labels.items) [index (get labels 0)]))

In [39]:
(defn assign-fields [tickets rules]
  (setv tickets (tuple tickets))
  (setv possible-labels (defaultdict list))
  (for [index (-> (first tickets) (len) (range))]
    (for [[label ranges] (rules.items)]
      (when (all-within (gfor ticket tickets (get ticket index)) ranges)
        (.append (get possible-labels index) label))))
  (narrow-fields possible-labels))

In [30]:
(setv TEST-RULES {"class" (, (Range 0 1) (Range 4 19))
                  "row" (, (Range 0 5) (Range 8 19))
                  "seat" (, (Range 0 13) (Range 16 19))})

In [31]:
(setv TEST-TICKETS (, (, 3 9 18) (, 15 1 5) (, 5 14 9)))

In [40]:
(assign-fields TEST-TICKETS TEST-RULES)

{0: 'row', 1: 'class', 2: 'seat'}

In [41]:
(setv label-map (assign-fields (valid-tickets TICKETS (flatten (RULES.values))) RULES))

In [42]:
(defn print-my-ticket [my-ticket label-map]
  (for [[index value] (enumerate my-ticket)] (print (get label-map index) value)))

In [43]:
(print-my-ticket MY-TICKET label-map)

duration 191
departure track 89
departure station 73
arrival platform 139
zone 71
departure location 103
class 109
train 53
arrival track 97
route 179
price 59
departure time 67
row 79
arrival location 101
wagon 113
departure platform 157
type 61
arrival station 107
seat 181
departure date 137


In [45]:
(reduce * (gfor [index label] (label-map.items) :if (label.startswith "departure") (get MY-TICKET index)))

964373157673